In [2]:
# Import the necessary modules
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from torchvision.models import resnet152, ResNet152_Weights
import os

In [ ]:
# Place the model on the GPU (if available)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if device.type == 'cuda':
    print(torch.cuda.get_device_name(device))
else:
    print(device)

GeForce 920MX


In [ ]:
# Step 1: Initialize model with the best available weights
weights = ResNet152_Weights.IMAGENET1K_V2
model = resnet152(weights=weights)

In [ ]:
# send your model weights to the GPU
if torch.cuda.is_available():
    model.cuda()
    model.to(torch.device('cuda'))
    print ("Model weights are on the GPU")

# Check if CUDA is available
if torch.cuda.is_available():
    # Use the GPU
    device = torch.device("cuda")
    print (device)
else:
    # Use the CPU
    device = torch.device("cpu")
    print (device)

Model weights are on the GPU
cuda


In [ ]:
# Define the loss function and the optimizer

# Cross entropy loss function for multi-class classification
criterion = nn.CrossEntropyLoss() 
# A Method for Stochastic Optimization
optimizer = torch.optim.Adam(model.parameters(), lr=0.04)

In [ ]:
# Define the file path to save the model
# PATH = "D:\Mamad\S.B.UK.AC.IR\Docs\M.Eng.in.ST\Courses\Deep.Learning\HomeWorks\HW03\model.cifar10.pth"
# PATH = "model.pth"

# Define the Directory path to save the Dataset
PATH_DIR_DATA = "D:\Mamad\S.B.UK.AC.IR\Docs\M.Eng.in.ST\Courses\Deep.Learning\HomeWorks\HW03\data\CIFAR10"

# Check if the file exists
# if os.path.isfile(PATH):
    # Load the saved model state dictionary
#     model.load_state_dict(torch.load(PATH))
#     print("Loaded model from:", PATH)

# else:
#     print("No saved model found. Starting with a new model.")

# Define the transforms for pre-processing and data augmentation
transform = transforms.Compose([
    transforms.ToTensor(), # Convert the data into tensor
    # transforms.RandomRotation(90), # Rotate the data randomly by 90 degrees
    # transforms.Normalize((0,), (0.5,))
])

# Load the CIFAR10 dataset
cifar10_data = datasets.CIFAR10(root=PATH_DIR_DATA, download=True, transform=transform)

# Split the dataset into train, test and cross validation sets
train_size = int(0.8 * len(cifar10_data)) # 80% of the data for training
test_size = int(0.1 * len(cifar10_data)) # 10% of the data for testing
val_size = len(cifar10_data) - train_size - test_size # 10% of the data for cross validation
train_data, test_data, val_data = torch.utils.data.random_split(cifar10_data, [train_size, test_size, val_size])

# Create data loaders for the train, validation and test sets
train_loader = DataLoader(train_data, batch_size=32, shuffle=True) # Batch size of 64 and shuffle the data
val_loader = DataLoader(val_data, batch_size=32, shuffle=False) # Batch size of 64 and do not shuffle the data
test_loader = DataLoader(test_data, batch_size=32, shuffle=False) # Batch size of 64 and do not shuffle the data

Files already downloaded and verified


In [ ]:
# Set the model to evaluation mode
model.eval()
# Initialize the test loss and accuracy
test_loss = 0.0
test_acc = 0.0
# Loop over the test batches
with torch.no_grad(): # No need to track gradients for testing
    for images, labels in test_loader:
        # Move the data to the device 
        images = images.to(device)
        labels = labels.to(device)
        # Forward pass
        outputs = model(images)
        # Calculate the loss
        loss = criterion(outputs, labels)
        # Update the test loss and accuracy
        test_loss += loss.item()
        test_acc += (outputs.argmax(dim=1) == labels).sum().item()
# Calculate the average test loss and accuracy
test_loss = test_loss / len(test_loader)
test_acc = test_acc / len(test_data)
# Print the test statistics
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {100*test_acc:.4f}%')


Test Loss: 58.0870, Test Accuracy: 0.0600%


In [ ]:
# Get the total number of parameters
total_params = sum(p.numel() for p in model.parameters())

# Get the total number of trainable parameters
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

# Print the total number and trainable parameters
print(f' Total Params: {total_params:.4f}, Trainable Params: {trainable_params:.4f}') 

 Total Params: 60192808.0000, Trainable Params: 60192808.0000


In [ ]:
classes = [
    "airplane",
    "automobile",
    "bird",
    "cat",
    "deer",
    "dog",
    "frog",
    "horse",
    "ship",
    "truck",
]

model.eval()
num = 8
images, labels = test_data[num][0], test_data[num][1]
with torch.no_grad():
    pred = model(images.unsqueeze(0))
    predicted, actual = classes[pred[0].argmax(0)], classes[labels]
    
    # Plot the image
    plt.imshow(images.permute(1, 2, 0))
    plt.title(f'Predicted: "{predicted}", Actual: "{actual}"')
    plt.show()


ModuleNotFoundError: No module named 'matplotlib'